### Coding Self-Attention in PyTorch!!!

---- 

In this tutorial, we will code **Self-Attention** in **[PyTorch](https://pytorch.org/)**. **Attention** is an essential component of neural network **Transformers**, which are driving the current excitement in **Large Language Models** and **AI**. Specifically, an **Enecoder-Only Transformer**, illustrated below, is the foundation for the popular model **BERT**. 

<img src="./images/encoder_only_1.png" alt="an enecoder-only transformer neural network" style="width: 800px;">

At the heart of **BERT** is **Self-Attention**, which allows it to establish relationships among the words, characters and symbols, that are used for input and collectively called **Tokens**. For example, in the illustration below, where the word **it** could potentially refer to either **pizza** or **oven**, **Attention** could help a **Transformer** establish the correctly relationship between the word **it** and **pizza**.

<img src="./images/attention_ex_1.png" alt="an illustration of how attention works" style="width: 800px;"/>

In this tutorial, you will...

- **[Code a Basic Self-Attention Class!!!](#selfAttention)** The basic self-attention class allows the transformer to establish relationships among words and tokens.

- **[Calculate Self-Attention Values!!!](#calculate)** We'll then use the class that we created, SelfAttention, to calculate self-attention values for some sample data.
 
- **[Verify The Calculations!!!](#validate)** Lastly, we'll validate the calculations made by the SelfAttention class..


In [ ]:
import torch # For tensors
import torch.nn as nn # For neural network modules and Linear class and bunch of other helper functions
import torch.nn.functional as F # For functions like softmax and relu.


In [ ]:
# This self Attention is inherited from nn.Module class. This is a class for self attention mechanism. 
# nn.Module is the base class for all neural network modules in pytorch. 
# Your models should also subclass this class.
class SelfAttention(nn.Module):
# First think we wpi;d is make a __init__ method. This is the constructor of the class and passing the
# In this method we are passing d_model the dimension of the model or the number of word embeddings values per token.
# d_model is used to define the Weight matrices that we'll use to create the Queries, Keys and Values.
    def __init__(self, d_model=2, # we are using 2 encoded values for each token. 2*2 weight matrices.
                row_dim = 0, # 
                col_dim = 1):
        # Next we call the parents __init__ method. This is done to initialize the parent class.
        super().__init__()
        # Now in order to creare the weight matrices for the Queries, Keys and Values we use the nn.Linear class. And why we use Linear class?
        # It is because we are going to use the dot product of the Queries and Keys to get the attention scores. 
        # So we need to have the same dimension for the Queries and Keys.
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False) 
        # Bias is set to False because we don't want any bias in the attention scores. It's not in the original Transformer paper.
                   # in_features is the number of rows in the weight matrix. Here it is d_model.
                   # defines the number of columns in the Weight matrix. Here it is d_model.
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = row_dim # This is the row index.
        self.col_dim = col_dim # This is the column index.
    # Now we add forward() method to the SelfAttention class. This method is called when we pass the input to the class object.
    # This is were we acually implement Self-Attention values for each token.and we are passing token encoding which is 
    # the Word Embeddings Plus Positional Encodings for each input token.
    def forward(self, token_encodings):
        # We are going to get the Queries, Keys and Values by multiplying the token_encodings with the Weight matrices.
        q = self.W_q(token_encodings) # Queries returns to the variable q.
        k = self.W_k(token_encodings) # Keys returns to the variable k.
        v = self.W_v(token_encodings)

        sims = torch.matmul(q,k.transpose(dim0 = self.row_dim, dim1 = self.col_dim)) # This is the dot product of Queries and Keys.
        # This is to calculate the similarity scores between all possible combinations of the Queries and Keys.
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5) # This is the scaled similarity scores.
        # This is done to avoid the exploding gradients problem.
        # The next thing we do is to apply the softmax function to the scaled similarity scores.
        attention_percents = F.softmax(scaled_sims, dim = self.col_dim) # Applying the softmax function to the scaled similarity scores
        # determine the percentages of influence that each token has on the other tokens.
        # Now we multiply the attention percentages with the Values to get the attention scores.
        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores # Finally we return the attention scores.

In [ ]:
encoding_matrix = torch.tensor([[1.16,0.23],
                                [0.56,0.78],
                                [0.12,0.34],
                                [0.89,0.67]])

torch.manual_seed(42) # This is to set the seed for the random number generator so that we hopefully get same results each time we run the code.

self_attention = SelfAttention(d_model=2, 
                               row_dim=0,
                               col_dim=1) # Creating an object of the SelfAttention class that has initialized weight matrices that
# are used to calculate the Queries, Keys and Values.

self_attention(encoding_matrix) # Passing the encoding_matrix to the self_attention object to get the attention scores.
# This last bit of the tensor is used for training the Weights with the help of the Backpropagation algorithm.

In [ ]:
# To validate that the math was done corretly, we can start by printing out the Wieghts in the maxtrix we use
# to calculate the Queries by transposing the weight property associated with the W_q object.

self_attention.W_q.weight.transpose(0,1) # This is the weight matrix that is used to calculate the Queries. 
# weight property is used to get the weight matrix and transpose() is used to transpose the matrix.
self_attention.W_k.weight.transpose(0,1) # This is the weight matrix that is used to calculate the Keys.
self_attention.W_v.weight.transpose(0,1) # This is the weight matrix that is used to calculate the Values.

# calculate the queries
self_attention.W_q(encoding_matrix) # This is the Queries.
# calculate the keys
self_attention.W_k(encoding_matrix) # This is the Keys.
# calculate the values
self_attention.W_v(encoding_matrix) # This is the Values.

q = self_attention.W_q(encoding_matrix) # This is the Queries.
k = self_attention.W_k(encoding_matrix) # This is the Keys.
v = self_attention.W_v(encoding_matrix) # This is the Values.

sims = torch.matmul(q,k.transpose(0,1)) # This is the dot product of the Queries and Keys.

scaled_sims = sims / (torch.tensor(2)**0.5) # This is the scaled similarity scores.
attention_percents = F.softmax(scaled_sims, dim=1) # This is the attention percentages.
attention_scores = torch.matmul(attention_percents, v) # This is the attention scores.